In [1]:
from polylearn import FactorizationMachineRegressor

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings

def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


from scipy import stats
from scipy.stats import norm, skew #for some statistics


In [3]:
# fm = FactorizationMachineRegressor()

In [4]:
train = pd.read_csv('../input/train_gen_feature.csv')
test = pd.read_csv('../input/test_gen_feature.csv')

In [5]:
y_train = np.load('y_train.npy')

In [40]:
test_id = pd.read_csv('../input/test.csv')['Id']

In [6]:
# y_train

In [7]:
# fm.fit(train, y_train)

In [8]:
# from sklearn.model_selection import cross_val_score

In [9]:
# cross_val_score(estimator=FactorizationMachineRegressor(), cv=3, X=train, y=y_train, n_jobs=-1, scoring='neg_mean_squared_error')

In [107]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC, Ridge
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [11]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [75]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))


In [135]:
ridge = make_pipeline(RobustScaler(), Ridge(alpha=6, random_state=1))

In [111]:
from sklearn.model_selection import GridSearchCV

In [132]:
params = {"alpha": [5.5, 6, 6.5, 7]}
gs = GridSearchCV(estimator=r, param_grid=params, cv=3, scoring='neg_mean_squared_error')

In [133]:
gs.fit(train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': [5.5, 6, 6.5, 7]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [134]:
gs.best_params_

{'alpha': 6}

In [77]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=1.0, random_state=3))


In [79]:
KRR = KernelRidge(alpha=1.0, kernel='polynomial', degree=2, coef0=2.5)


In [81]:
GBoost = GradientBoostingRegressor(n_estimators=4000, learning_rate=0.05,
                                   max_depth=3, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [83]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.7, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=3000,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [84]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)


In [18]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
cross_val_score(estimator=lasso, cv=3, X=trin_pn, y=y_train, n_jobs=-1, scoring='neg_mean_squared_error')

# base model score

In [136]:
score = rmsle_cv(ridge)
print(score.mean())

0.11439351383


In [76]:
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Lasso score: 0.1115 (0.0074)



In [78]:
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

ElasticNet score: 0.1115 (0.0074)



In [80]:
score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Kernel Ridge score: 0.1149 (0.0070)



In [82]:
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Gradient Boosting score: 0.1164 (0.0085)



In [85]:
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Xgboost score: 0.1162 (0.0063)



In [24]:
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

LGBM score: 0.1156 (0.0069)



# stack model

In [28]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)   

In [29]:
averaged_models = AveragingModels(models = (ENet, GBoost, KRR, lasso))


In [30]:
score = rmsle_cv(averaged_models)


In [32]:
print(score.mean(),  score.std())

(0.10866239908742363, 0.0076943592036042181)


In [34]:
averaged_models.fit(train, y_train)

AveragingModels(models=(Pipeline(steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('elasticnet', ElasticNet(alpha=0.0005, copy_X=True, fit_intercept=True, l1_ratio=0.9,
      max_iter=1000, normalize=False, positive=False, precompute=False...ve=False, precompute=False, random_state=1,
   selection='cyclic', tol=0.0001, warm_start=False))])))

In [35]:
predictions = averaged_models.predict(test)

In [37]:
predictions = np.expm1(predictions)

In [41]:
result = pd.DataFrame({'Id': test_id, 'SalePrice': predictions})

In [44]:
result.to_csv('./result_average.csv', index=False)

In [86]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [137]:
stacked_averaged_models = StackingAveragedModels(base_models = (ridge, ENet, GBoost, KRR),
                                                 meta_model = lasso)

score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

Stacking Averaged models score: 0.1085 (0.0073)


In [ ]:
1083

In [88]:
stacked_averaged_models.fit(train.values, y_train)

StackingAveragedModels(base_models=(Pipeline(steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('lasso', Lasso(alpha=0.0005, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=1,
   s...nelRidge(alpha=1.0, coef0=2.5, degree=2, gamma=None, kernel='polynomial',
      kernel_params=None)),
            meta_model=Pipeline(steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('lasso', Lasso(alpha=0.0005, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=1,
   selection='cyclic', tol=0.0001, warm_start=False))]),
            n_folds=5)

In [89]:
predictions = stacked_averaged_models.predict(test)

In [90]:
predictions = np.expm1(predictions)

In [91]:
result = pd.DataFrame({'Id': test_id, 'SalePrice': predictions})

In [92]:
result.to_csv('./result_stack.csv', index=False)

In [94]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [138]:
stacked_averaged_models.fit(train.values, y_train)
stacked_train_pred = stacked_averaged_models.predict(train.values)
stacked_pred = np.expm1(stacked_averaged_models.predict(test.values))
print(rmsle(y_train, stacked_train_pred))

0.0788125985792


In [96]:
model_xgb.fit(train, y_train)
xgb_train_pred = model_xgb.predict(train)
xgb_pred = np.expm1(model_xgb.predict(test))
print(rmsle(y_train, xgb_train_pred))


0.0753738107833


In [97]:
model_lgb.fit(train, y_train)
lgb_train_pred = model_lgb.predict(train)
lgb_pred = np.expm1(model_lgb.predict(test.values))
print(rmsle(y_train, lgb_train_pred))


0.0724680051983


In [139]:
print('RMSLE score on train data:')
print(rmsle(y_train,stacked_train_pred*0.60 +
               xgb_train_pred*0.2 + lgb_train_pred*0.2 ))

RMSLE score on train data:
0.0743981576816


In [140]:
ensemble = stacked_pred*0.60 + xgb_pred*0.2 + lgb_pred*0.2


In [141]:
result = pd.DataFrame({'Id': test_id, 'SalePrice': ensemble})

In [142]:
result.to_csv('./ensemble.csv', index=False)

In [143]:
result.head()

,Id,SalePrice
0,1461,120463.883294
1,1462,161165.590695
2,1463,187830.862220
3,1464,196490.822839
4,1465,191702.680488
